# 필요 구성 라이브러리 다운로드

In [38]:
# !pip install --upgrade pip
# !pip install konlpy
# !pip install nltk
# !pip install sklearn
# !pip install pandas
# !pip install jellyfish
# !pip install segtok
# !pip install networkx
# !pip install matplotlib
# !pip install tensorflow
# !pip install krwordrank
# !pip install soynlp
# !pip install textrank
# !pip install sentence_transformers
# !pip install spaCy
# !python -m spacy download ko_core_news_sm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2022-10-21 09:38:12.181760: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 11.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ko_core_news_sm')


# 텍스트 마이닝 실험

In [36]:
# 필요 라이브러리 선언
from pprint import pprint

from yake import korea_token
from yake.korea_token import edit_josa, edit_sentences
import math

from konlpy.tag import Komoran
from collections import Counter

import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from math import log
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
np.random.seed(seed=0)

from krwordrank.word import KRWordRank
from krwordrank.sentence import summarize_with_sentences
from krwordrank.word import summarize_with_keywords

from scipy.sparse import csr_matrix
from soynlp.tokenizer import MaxScoreTokenizer
from sklearn.metrics import pairwise_distances

import networkx

# from textrank import KeywordSummarizer

import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import spacy
from spacy.lang.ko.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [12]:
komoran = Komoran()

In [2]:
STRTexT = "러시아가 우크라이나 키이우 등을 공습한 것은 군 내부 비판과 블라디미르 푸틴 러시아 대통령의 자존심 때문이라는 분석이 나왔다. 영국 가디언지는 10일(현지시간) 푸틴 대통령의 대규모 공습은 국내 군 비판세력, 러시아가 침공에서 실패하고 있다는 사실, 크림대교 폭발 후 상처받은 자존심에 대한 절박한 답변이라고 풀이했다. 가디언에 따르면 미국 싱크탱크 카네기국제평화재단의 선임 연구원 안드레이 콜레스니코프는 '지금 푸틴이 하는 것은 사소한 복수'라며 '개인적 복수도 있다'고 말했다. 러시아 전쟁 전문가와 군사 블로거 등은 수개월간 우크라이나를 상대로 전면전을 벌이라고 촉구해왔고 키이우 등의 거리에 시신이 있는 끔찍한 사진이 나오자 지금은 만족해한다. 최근 러시아군 수뇌부를 비판했던 람잔 카디로프 체첸 공화국 수장은 '(볼로디미르) 젤렌스키(우크라이나 대통령), 우리는 러시아가 아직 제대로 시작도 안했다고 경고했다'라며 '이제 전쟁 진행에 100% 만족한다'고 말했다. 푸틴 대통령은 이날 러시아 안전보장이사회 회의를 주재하고 크림대교 폭발의 배후로 우크라이나 정보 당국을 겨냥하면서 '이런 종류의 범죄에 대응을 하지 않는 것은 불가능하다'고 말했다. 그러나 드미트로 쿨레바 우크라이나 외무장관은 '러시아는 크림대교 사건 전에도 우크라이나에 계속 미사일 공격을 했다'며 '푸틴은 전투 패배로 절박한 상황이며 전황을 유리하게 바꾸려고 미사일 공포를 사용한다'고 반박했다. 우크라이나 구조대원들이 10일(현지시간) 수도 키이우 시내에서 러시아군의 미사일 공격을 당한 현장을 조사하고 있다. 이날 오전에 키이우 시내에 여러차례 폭발이 발생했다. 푸틴 대통령은 이번 공격이 국방부의 요청에 따라 이뤄졌다고 주장했는데, 이것이 사실이라면 이는 새로운 합동군 총사령관 세르게이 수로비킨의 첫번째 결정이라고 가디언은 전했다. 수로비킨 사령관과 함께 일했던 전 국방부 관계자는 가디언지에 '오늘 키이우에서 벌어진 일이 놀랍지 않았다. 그는 매우 무자비하고 사람 목숨을 신경 쓰지 않는다'며 '그의 손이 우크라이나인의 피로 뒤덮일까 걱정된다'고 말했다. 그러나 푸틴 대통령이 이날 공습으로 얻은 매파들의 호평은 오래가지 않을 수 있다. 그는 이번과 같은 대규모 미사일 공격은 러시아 영토 공격 시 대응으로 남겨둘 것이라고 말했는데 강경파들은 전면전을 원하고 있다. 정치 평론가인 세르게이 마르코프는 '러시아 여론은 대규모 공격과 우크라이나군이 사용할 가능성이 있는 인프라 완전파괴를 원한다'고 말했다. 한편으론 비판 세력의 목소리는 그다지 의미가 없을 수도 있어 보인다. 전쟁과 관련한 의사결정 과정은 여전히 불투명하다. 콜레스니코프 선임 연구원은 '푸틴 대통령으로선 매파와 극보수파의 불만에 대응하는 것이 중요하겠지만 나는 그들의 영향력을 과장하진 않을 것'이라며 '푸틴 자신이 가장 매파적이고 극보수적인 인물'이라고 말했다. 러시아에서 나오는 한 가지 이론은 푸틴 대통령이 악명 높은 새로운 군사령관을 임명함으로써 전쟁에서 국방부의 성과에 대한 분노를 줄이려고 한다는 것이다. 수로비킨 사령관과 2020년까지 함께 일했던 전 공군 중위인 글렙 이리소프는 '수로비킨은 강경파들을 선호하고 와그너 용병회사와도 좋은 관계를 유지한다'며 '그러나 그가 매우 잔인한 동시에 유능한 사령관이지만 모든 문제를 풀 수는 없을 것'이라고 말했다."
LISTText = ["러시아가 우크라이나 키이우 등을 공습한 것은 군 내부 비판과 블라디미르 푸틴 러시아 대통령의 자존심 때문이라는 분석이 나왔다.",
            "영국 가디언지는 10일(현지시간) 푸틴 대통령의 대규모 공습은 국내 군 비판세력, 러시아가 침공에서 실패하고 있다는 사실, 크림대교 폭발 후 상처받은 자존심에 대한 절박한 답변이라고 풀이했다.",
            "가디언에 따르면 미국 싱크탱크 카네기국제평화재단의 선임 연구원 안드레이 콜레스니코프는 '지금 푸틴이 하는 것은 사소한 복수'라며 '개인적 복수도 있다'고 말했다.",
            "러시아 전쟁 전문가와 군사 블로거 등은 수개월간 우크라이나를 상대로 전면전을 벌이라고 촉구해왔고 키이우 등의 거리에 시신이 있는 끔찍한 사진이 나오자 지금은 만족해한다.",
            "최근 러시아군 수뇌부를 비판했던 람잔 카디로프 체첸 공화국 수장은 '(볼로디미르) 젤렌스키(우크라이나 대통령), 우리는 러시아가 아직 제대로 시작도 안했다고 경고했다'라며 '이제 전쟁 진행에 100% 만족한다'고 말했다.",
            "푸틴 대통령은 이날 러시아 안전보장이사회 회의를 주재하고 크림대교 폭발의 배후로 우크라이나 정보 당국을 겨냥하면서 '이런 종류의 범죄에 대응을 하지 않는 것은 불가능하다'고 말했다.",
            "그러나 드미트로 쿨레바 우크라이나 외무장관은 '러시아는 크림대교 사건 전에도 우크라이나에 계속 미사일 공격을 했다'며 '푸틴은 전투 패배로 절박한 상황이며 전황을 유리하게 바꾸려고 미사일 공포를 사용한다'고 반박했다.",
            "우크라이나 구조대원들이 10일(현지시간) 수도 키이우 시내에서 러시아군의 미사일 공격을 당한 현장을 조사하고 있다.",
            "이날 오전에 키이우 시내에 여러차례 폭발이 발생했다.",
            "푸틴 대통령은 이번 공격이 국방부의 요청에 따라 이뤄졌다고 주장했는데, 이것이 사실이라면 이는 새로운 합동군 총사령관 세르게이 수로비킨의 첫번째 결정이라고 가디언은 전했다.",
            "수로비킨 사령관과 함께 일했던 전 국방부 관계자는 가디언지에 '오늘 키이우에서 벌어진 일이 놀랍지 않았다.",
            "그는 매우 무자비하고 사람 목숨을 신경 쓰지 않는다'며 '그의 손이 우크라이나인의 피로 뒤덮일까 걱정된다'고 말했다.",
            "그러나 푸틴 대통령이 이날 공습으로 얻은 매파들의 호평은 오래가지 않을 수 있다.",
            "그는 이번과 같은 대규모 미사일 공격은 러시아 영토 공격 시 대응으로 남겨둘 것이라고 말했는데 강경파들은 전면전을 원하고 있다.",
            "정치 평론가인 세르게이 마르코프는 '러시아 여론은 대규모 공격과 우크라이나군이 사용할 가능성이 있는 인프라 완전파괴를 원한다'고 말했다.",
            "한편으론 비판 세력의 목소리는 그다지 의미가 없을 수도 있어 보인다.",
            "전쟁과 관련한 의사결정 과정은 여전히 불투명하다.",
            "콜레스니코프 선임 연구원은 '푸틴 대통령으로선 매파와 극보수파의 불만에 대응하는 것이 중요하겠지만 나는 그들의 영향력을 과장하진 않을 것'이라며 '푸틴 자신이 가장 매파적이고 극보수적인 인물'이라고 말했다.",
            "러시아에서 나오는 한 가지 이론은 푸틴 대통령이 악명 높은 새로운 군사령관을 임명함으로써 전쟁에서 국방부의 성과에 대한 분노를 줄이려고 한다는 것이다.",
            "수로비킨 사령관과 2020년까지 함께 일했던 전 공군 중위인 글렙 이리소프는 '수로비킨은 강경파들을 선호하고 와그너 용병회사와도 좋은 관계를 유지한다'며 '그러나 그가 매우 잔인한 동시에 유능한 사령관이지만 모든 문제를 풀 수는 없을 것'이라고 말했다."
        ]


# new_text = edit_sentences(STRTexT)
# # print('원본 split_sentences :', new_text)
# total_value = []
# for x in range(len(new_text)):
#     total_value.append(' '.join(edit_josa(new_text[x])))

# print(total_value)
TUNNEDText = ['러시아 우크라이나 키이우 등 공습 것 군 내부 비판 블라디미르 푸틴 러시아 대통령 자존심 때문 분석 나오',
              '영국 가디언지 10일 현지시간 푸틴 대통령 대규모 공습 국내 군 비판세력 러시아 침공 실패 있 사실 크림대교 폭발 후 상처 자존심 대하 절박 답변 풀이',
              '가디언 따르 미국 싱크탱크 카네기국제평화재단 선임 연구원 안드레이 콜레스니코프 지금 푸틴 하 것 사소 복수 이 개인 복수 있 고 말',
              '러시아 전쟁 전문가 군사 블로거 등 수개월간 우크라이나 상대로 전면전 벌이 촉구 키이우 등 거리 시신 있 끔찍 사진 나오 지금 만족',
              '최근 러시아군 수뇌부 비판 람잔 카디로프 체첸 공화국 수장 보 젤렌스 우크라이나 대통령 우리 러시아 아직 제대로 시작 안 경고 이 이제 전쟁 진행 100 만족 고 말',
              '푸틴 대통령 이날 러시아 안전보장 회의 주재 크림대교 폭발 배후 우크라이나 정보 당국 겨냥 이런 종류 범죄 대응 하 않 것 불가능 고 말',
              '그러나 드미트 쿨레바 우크라이나 외무장관 러시아 크림대교 사건 전 우크라이나 계속 미사일 공격 하 이 푸틴 전투 패배 절박 상황 전황 유리 바꾸 미사일 공포 사용 고 반박',
              '우크라이나 구조대원 10일 현지시간 수 키이우 시내 러시아군 미사일 공격 당하 현장 조사 있',
              '이날 오전 키이우 시내 여러차례 폭발 발생', 
              '푸틴 대통령 이번 공격 국방부 요청 따르 이루어지 주장 이것 사실 이 새롭 합동군 총사령관 세르 수로비킨 첫번 결정 가디언 전하', 
              '수로비킨 사령관 함께 일 전 국방부 관계자 가디언지 오늘 키이우 벌어지 일 놀랍 않', 
              '그 매우 무자비 사람 목숨 신경 쓰 않 이 그 손 우크라이나인 피로 뒤덮이 걱정 고 말', 
              '그러나 푸틴 대통령 이날 공습 얻 매파 호평 오래가 않 수 있', 
              '그 이번 같 대규모 미사일 공격 러시아 영토 공격 시 대응 남기 것 말 강경파 전면전 원하 있', 
              '정치 평론가 세르 마르코프 러시아 여론 대규모 공격 우크라이나군 사용 가능 있 인프라 완전파괴 원하 고 말', 
              '한편 비판 세력 목소리 그다지 의미 없 수 있 보이', 
              '전쟁 관련 의사결정 과정 여전히 불투명', 
              '콜레스니코프 선임 연구원 푸틴 대통령 매파 극보수파 불만 대응 것 중요 나 그 영향력 과장하진 않 것 이 푸틴 자신 가장 매파 극보수 인물 이 말', 
              '러시아 나오 한 가지 이론 푸틴 대통령 악명 높 새롭 군사령관 임명 전쟁 국방부 성과 대하 분노 줄이 하 것', 
              '수로비킨 사령관 2020년 함께 일 전 공군 중위 글렙 이리소프 수로비킨 강경파 선호 오 용병회사 좋 관계 유지 이 그러나 그 매우 잔인 동시 유능 사령관 모든 문제 풀 수 없 것 이 말']

빈도수 기반 명사 추출

In [3]:
# STRTexT에서 명사를 추출, 명사의 수를 count로 선언
noun = komoran.nouns(STRTexT)
count = Counter(noun)

# 명사 빈도 카운트
noun_list = count.most_common(20)
for v in noun_list:
    if len(v[0]) == 1:
        pass
    else:
        print("Komoran 기준 : " + str(v))

Komoran 기준 : ('러시아', 10)
Komoran 기준 : ('푸틴', 9)
Komoran 기준 : ('우크라이나', 8)
Komoran 기준 : ('대통령', 8)
Komoran 기준 : ('공격', 6)
Komoran 기준 : ('비판', 4)
Komoran 기준 : ('가디언', 4)
Komoran 기준 : ('전쟁', 4)
Komoran 기준 : ('미사일', 4)
Komoran 기준 : ('수로', 4)
Komoran 기준 : ('비킨', 4)
Komoran 기준 : ('이우', 3)
Komoran 기준 : ('공습', 3)


TF.IDF 

In [123]:
vectorizer = TfidfVectorizer()
vectorizer.fit(TUNNEDText)
matrix = vectorizer.transform(TUNNEDText)

vocabulary_word_id = defaultdict(int)
    
for idx, token in enumerate(vectorizer.get_feature_names()):
    vocabulary_word_id[token] = idx
    
# 특징 추출 결과: {"token": value}
result = defaultdict(str)
    
for token in vectorizer.get_feature_names():
    result[token] = matrix[0, vocabulary_word_id[token]]
    
# 내림차순 (중요도 high) 기준 정렬
result = sorted(result.items(), key = lambda item: item[1], reverse = True)
pprint(result[:20])

[('러시아', 0.34349799588831337),
 ('내부', 0.33043401009471474),
 ('때문', 0.33043401009471474),
 ('분석', 0.33043401009471474),
 ('블라디미르', 0.33043401009471474),
 ('자존심', 0.2904565526794969),
 ('공습', 0.2620920952438779),
 ('나오', 0.2620920952438779),
 ('비판', 0.2620920952438779),
 ('키이우', 0.22211463782865998),
 ('우크라이나', 0.20691591390621303),
 ('대통령', 0.1821371804134421),
 ('푸틴', 0.17174899794415668),
 ('100', 0.0),
 ('10일', 0.0),
 ('2020년', 0.0),
 ('가능', 0.0),
 ('가디언', 0.0),
 ('가디언지', 0.0),
 ('가장', 0.0)]


In [125]:
countList = []

tfidfv = TfidfVectorizer().fit(TUNNEDText)

for k,v in tfidfv.vocabulary_.items():
    textTuple = ((k, v))
    countList.append(textTuple)

countList.sort(reverse=True)
countList[:20]

[('회의', 197),
 ('호평', 196),
 ('현지시간', 195),
 ('현장', 194),
 ('합동군', 193),
 ('함께', 192),
 ('한편', 191),
 ('피로', 190),
 ('풀이', 189),
 ('푸틴', 188),
 ('폭발', 187),
 ('평론가', 186),
 ('패배', 185),
 ('키이우', 184),
 ('크림대교', 183),
 ('쿨레바', 182),
 ('콜레스니코프', 181),
 ('카디로프', 180),
 ('카네기국제평화재단', 179),
 ('침공', 178)]

Text Summarization

In [40]:
# 원 예제는 영어를 입력 하면 번역 후 한국어로 출력되는 기능이 있음.
# 이번에는 한국어를 입력했기 때문에 해당 기능이 사용되지 않음. (영어 입력하면 한국어 나오는지는 실험 하지 않음.)

# 사용된 방식은 추상적 방법론을 사용해 만든 로직임. 추상적 방법론이란, 대상 문서의 의미를 해석하여 지능적으로 요약문을 만드는 방법으로, 완전히 새로운 문장을 생성함.
# 다른 방식으로는 대상 문서에서 중요한 문장을 식별하여 요약문을 만드는 방법인 추출식 방법론이 있음.

def summarize(text, per):
    nlp = spacy.load('ko_core_news_sm')
    doc = nlp(text)
    tokens = [token.text for token in doc]
    word_frequencies = {}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
    max_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word]/max_frequency
    sentence_tokens = [sent for sent in doc.sents]
    sentence_score = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_score.keys():
                    sentence_score[sent] = word_frequencies[word.text.lower()]
                else:
                    sentence_score [sent] += word_frequencies[word.text.lower()]

    select_length = int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_score, key=sentence_score.get)
    final_summary = [word.text for word in summary]
    summary = ''.join(final_summary)
    return summary

summarized_text = summarize(STRTexT, 0.1)
print('10% 요약 결과: ', summarized_text)

10% 요약 결과:  최근 러시아군 수뇌부를 비판했던 람잔 카디로프 체첸 공화국 수장은 '(볼로디미르) 젤렌스키(우크라이나 대통령), 우리는 러시아가 아직 제대로 시작도 안했다고 경고했다'라며 '이제 전쟁 진행에 100% 만족한다'고 말했다.그러나 드미트로 쿨레바 우크라이나 외무장관은 '러시아는 크림대교 사건 전에도 우크라이나에 계속 미사일 공격을 했다'며 '푸틴은 전투 패배로 절박한 상황이며 전황을 유리하게 바꾸려고 미사일 공포를 사용한다'고 반박했다.


* Text Summarization With Attention

In [ ]:
# 해당 기능은 기계학습이 필요하기 때문에 후순위로 선정한다.

* TextRank Based on Sentence Embedding

In [ ]:
# 해당 기능은 기계학습이 필요하기 때문에 후순위로 선정한다.


# Glove를 활용한 

RAKE

WordRank

* KR-WordRank

In [110]:
texts = TUNNEDText
# texts = LISTText
wordrank_extractor = KRWordRank(min_count=5, max_length=10)
keywords, rank, graph = wordrank_extractor.extract(texts, num_keywords=100)

def make_vocab_score(keywords, scaling=None):
    if scaling is None:
        scaling = lambda x:math.sqrt(x)
    return {word:scaling(rank) for word, rank in keywords.items()}

keywords = make_vocab_score(keywords)
print(keywords)

{'푸틴': 1.3732243555746098, '대통령': 1.3521268801052138, '러시아': 1.3020997627605881, '우크': 1.1216525136431523, '공격': 1.0979590292080694, '키이': 1.0581310608472447}


* sentence extraction

In [112]:
texts = LISTText
keywords, sents = summarize_with_sentences(texts, num_keywords=100, num_keysents=10)
print(sents)

["푸틴 대통령은 이날 러시아 안전보장이사회 회의를 주재하고 크림대교 폭발의 배후로 우크라이나 정보 당국을 겨냥하면서 '이런 종류의 범죄에 대응을 하지 않는 것은 불가능하다'고 말했다.", "정치 평론가인 세르게이 마르코프는 '러시아 여론은 대규모 공격과 우크라이나군이 사용할 가능성이 있는 인프라 완전파괴를 원한다'고 말했다.", '영국 가디언지는 10일(현지시간) 푸틴 대통령의 대규모 공습은 국내 군 비판세력, 러시아가 침공에서 실패하고 있다는 사실, 크림대교 폭발 후 상처받은 자존심에 대한 절박한 답변이라고 풀이했다.', "가디언에 따르면 미국 싱크탱크 카네기국제평화재단의 선임 연구원 안드레이 콜레스니코프는 '지금 푸틴이 하는 것은 사소한 복수'라며 '개인적 복수도 있다'고 말했다.", '우크라이나 구조대원들이 10일(현지시간) 수도 키이우 시내에서 러시아군의 미사일 공격을 당한 현장을 조사하고 있다.', '푸틴 대통령은 이번 공격이 국방부의 요청에 따라 이뤄졌다고 주장했는데, 이것이 사실이라면 이는 새로운 합동군 총사령관 세르게이 수로비킨의 첫번째 결정이라고 가디언은 전했다.', "수로비킨 사령관과 2020년까지 함께 일했던 전 공군 중위인 글렙 이리소프는 '수로비킨은 강경파들을 선호하고 와그너 용병회사와도 좋은 관계를 유지한다'며 '그러나 그가 매우 잔인한 동시에 유능한 사령관이지만 모든 문제를 풀 수는 없을 것'이라고 말했다.", '이날 오전에 키이우 시내에 여러차례 폭발이 발생했다.', '한편으론 비판 세력의 목소리는 그다지 의미가 없을 수도 있어 보인다.', '전쟁과 관련한 의사결정 과정은 여전히 불투명하다.']


TextRank 

In [3]:
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
 
 
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

* 문장 요약

In [13]:
tr = TextRank()
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
tr.loadSents(RawSentenceReader('test.txt'), lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), komoran.pos(sent)))
# print('Build...')
tr.build()
ranks = tr.rank()
for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
print(tr.summarize(0.2))

0	0.08666798879227969	러시아가 우크라이나 키이우 등을 공습한 것은 군 내부 비판과 블라디미르 푸틴 러시아 대통령의 자존심 때문이라는 분석이 나왔다.
5	0.07029042057278401	 푸틴 대통령은 이날 러시아 안전보장이사회 회의를 주재하고 크림대교 폭발의 배후로 우크라이나 정보 당국을 겨냥하면서 '이런 종류의 범죄에 대응을 하지 않는 것은 불가능하다'고 말했다.
13	0.06815514561018991	 그는 이번과 같은 대규모 미사일 공격은 러시아 영토 공격 시 대응으로 남겨둘 것이라고 말했는데 강경파들은 전면전을 원하고 있다.
1	0.06429569851243916	 영국 가디언지는 10일(현지시간) 푸틴 대통령의 대규모 공습은 국내 군 비판세력, 러시아가 침공에서 실패하고 있다는 사실, 크림대교 폭발 후 상처받은 자존심에 대한 절박한 답변이라고 풀이했다.
18	0.061683938582030896	 러시아에서 나오는 한 가지 이론은 푸틴 대통령이 악명 높은 새로운 군사령관을 임명함으로써 전쟁에서 국방부의 성과에 대한 분노를 줄이려고 한다는 것이다.
12	0.061574888759916464	 그러나 푸틴 대통령이 이날 공습으로 얻은 매파들의 호평은 오래가지 않을 수 있다.
14	0.05919188596906117	 정치 평론가인 세르게이 마르코프는 '러시아 여론은 대규모 공격과 우크라이나군이 사용할 가능성이 있는 인프라 완전파괴를 원한다'고 말했다.
17	0.05865832561198442	 콜레스니코프 선임 연구원은 '푸틴 대통령으로선 매파와 극보수파의 불만에 대응하는 것이 중요하겠지만 나는 그들의 영향력을 과장하진 않을 것'이라며 '푸틴 자신이 가장 매파적이고 극보수적인 인물'이라고 말했다.
3	0.056361527263808664	 러시아 전쟁 전문가와 군사 블로거 등은 수개월간 우크라이나를 상대로 전면전을 벌이라고 촉구해왔고 키이우 등의 거리에 시신이 있는 끔찍한 사진이 나오자 지금은 만족해한다.
9	0.0547234582

* 키워드 추출

In [15]:
tr = TextRank(window=5, coef=1)
# print('Load...')
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV'), ('없', 'VV') ])
tr.load(RawTaggerReader('test.txt', tagger=komoran), lambda w: w not in stopword and (w[1] in ('NNG', 'NNP', 'VV', 'VA')))
# print('Build...')
tr.build()
kw = tr.extract(0.1)
for k in sorted(kw, key=kw.get, reverse=True):
    print("%s\t%g" % (k, kw[k]))

크림 대교 5.3471075307174685
수로 비킨 5.059425458265688
미사일 공격 4.366278277705742
푸틴 대통령 3.960813169597578
규모 미사일 3.960813169597578
군 비판 3.960813169597578
규모 공격 3.5553480614894135
우크라이나 군 3.2676659890376327
이날 러시아 3.044522437723423
러시아 전쟁 2.7568403652716422
우크라이나 키 2.7568403652716422
우크라이나 대통령 2.2868367360259065
러시아 대통령 2.063693184711697
(('푸틴', 'NNP'), ('대통령', 'NNG'))	0.156417
(('미사일', 'NNG'), ('공격', 'NNG'))	0.117627
(('수로', 'NNP'), ('비킨', 'NNP'))	0.104941
(('크림', 'NNP'), ('대교', 'NNP'))	0.104452
(('러시아', 'NNP'),)	0.10077
(('우크라이나', 'NNP'), ('키', 'NNG'))	0.0949822
(('군', 'NNG'), ('비판', 'NNG'))	0.0861607
(('규모', 'NNG'),)	0.0472933
(('매파', 'NNP'),)	0.0459871
(('이날', 'NNG'),)	0.0432415
(('전쟁', 'NNG'),)	0.0406895
(('국방부', 'NNP'),)	0.0389833
(('가디언', 'NNP'),)	0.0380954
(('말', 'NNG'),)	0.032053


TopicRank 

TopicalPageRank 

PositionRank 

MultipartiteRank  

ExpandRank 

KEA 

KeyBERT 

* Korean KeyBERT

In [25]:
from konlpy.tag import Okt

okt = Okt()
tokenized_doc = okt.pos(STRTexT)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

n_gram_range = (2, 3)
count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수: ', len(candidates))
print('trigram 5개 출력: ', candidates[:5])

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([STRTexT])
candidate_embeddings = model.encode(candidates)

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

품사 태깅 10개만 출력 : [('러시아', 'Noun'), ('가', 'Josa'), ('우크라이나', 'Noun'), ('키이우', 'Noun'), ('등', 'Noun'), ('을', 'Josa'), ('공습', 'Noun'), ('한', 'Josa'), ('것', 'Noun'), ('은', 'Josa')]
명사 추출 : 러시아 우크라이나 키이우 등 공습 것 군 내부 비판 블라디미르 푸틴 러시아 대통령 자존심 때문 분석 영국 가디언 현지 시간 푸틴 대통령 대규모 공습 국내 군 비판 세력 러시아 침공 사실 크림 대교 폭발 후 상처 자존심 대한 절박 답변 풀이 가디언 미국 싱크탱크 카네기 국제 평화 재단 선임 연구원 안드레이 콜레스니코프 지금 푸틴 것 복수 라며 개인 복수 고 말 러시아 전쟁 전문가 군사 블로거 등 개 월간 우크라이나 상대로 면전 벌이 촉구 키이우 등 거리 시신 사진 지금 최근 러시아군 수뇌부 비판 람잔 카디 로프 체첸 공화국 수장 볼 로디 미르 젤 렌스 키 우크라이나 대통령 우리 러시아 제대로 시작 경고 라며 이제 전쟁 진행 고 말 푸틴 대통령 날 러시아 안전보장 이사회 회의 주재 크림 대교 폭발 배후 우크라이나 정보 당국 겨냥 종류 범죄 대응 것 고 말 드미트 쿨 레바 우크라이나 외무 장관 러시아 크림 대교 사건 전 우크라이나 계속 미사일 공격 며 푸틴 전투 패배 절박 상황 전황 미사일 공포 사용 고 반박 우크라이나 구조대 현지 시간 수도 키이우 시내 러시아군 미사일 공격 당한 현장 조사 날 오전 키이우 시내 차례 폭발 발생 푸틴 대통령 이번 공격 국방부 요청 주장 것 합동 총사령관 세르게이 로비킨 첫 결정 가디언 전 로비킨 사령관 일 전 국방부 관계자 가디언 오늘 키이우 그 매우 사람 목숨 신경 며 그 손 우크라이나인 피로 뒤 걱정 고 말 푸틴 대통령 날 공습 매파 호평 가지 수 그 이번 대규모 미사일 공격 러시아 영토 공격 시 대응 둘 것 말 강경 파 면전 정치 평론가 세르게이 마르코프 러시아 여론 대규모 공격 우크라이나 사용 가능성 

Downloading: 100%|██████████| 190/190 [00:00<00:00, 82.5kB/s]
Downloading: 100%|██████████| 4.06k/4.06k [00:00<00:00, 1.72MB/s]
Downloading: 100%|██████████| 122/122 [00:00<00:00, 53.1kB/s]
Downloading: 100%|██████████| 1.11G/1.11G [00:19<00:00, 55.7MB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 23.2kB/s]
Downloading: 100%|██████████| 5.07M/5.07M [00:02<00:00, 2.32MB/s]
Downloading: 100%|██████████| 150/150 [00:00<00:00, 64.6kB/s]
Downloading: 100%|██████████| 9.10M/9.10M [00:00<00:00, 43.5MB/s]
Downloading: 100%|██████████| 229/229 [00:00<00:00, 101kB/s]


['영향력 라며 푸틴', '러시아 전쟁 전문가', '러시아군 수뇌부 비판', '공습 국내 비판', '비판 블라디미르 푸틴']


* Max Sum Similarity : 
데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됨.
후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 로직임.

In [33]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]


print(max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=6, nr_candidates=10))
print(max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=6, nr_candidates=30))  # nr_candidates가 높을수록 다양한 키워드 추출

['개인 복수 러시아', '걱정 푸틴 대통령', '공습 내부 비판', '영향력 라며 푸틴', '러시아 전쟁 전문가', '공습 국내 비판']
['내부 비판 블라디미르', '전쟁 국방부 성과', '푸틴 전투 패배', '미사일 공격 푸틴', '러시아 침공 사실', '공습 국내 비판']


* Maximal Marginal Relevance : 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하는 로직. 문서와 가장 유사한 키워드/키프레이즈를 선택한 후 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보를 반복적으로 선택함

In [29]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]


print(mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.2)) # 상대적으로 낮은 diversity값을 설정하면, 기존의 코사인 유사도만 사용한 것과 매우 유사함
print(mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.7)) # 상대적으로 높은 diversity값을 설정하면 다양한 키워드를 추출함

['비판 블라디미르 푸틴', '공습 국내 비판', '러시아 전쟁 전문가', '러시아군 수뇌부 비판', '개인 복수 러시아', '영향력 라며 푸틴', '비판 세력 러시아', '걱정 푸틴 대통령', '공습 내부 비판', '푸틴 대통령 공습']
['비판 블라디미르 푸틴', '대규모 공습', '전투 패배 절박', '평화 재단 선임', '거리 시신', '배후 우크라이나 정보', '전문가 군사 블로거', '러시아 침공 사실', '범죄 대응', '보수파 불만 대응']
